<pre>
<img align="center" width="300" src="https://raw.githubusercontent.com/andrelmfsantos/ppga/master/logo%20uninove.png">
</pre>

<pre>
<img align="center" width="150" src="https://camo.githubusercontent.com/bdc6a3b8963aa99ff57dfd6e1e4b937bd2e752bcb1f1936f90368e5c3a38f670/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f4c6963656e73652d434325323042592d2d5341253230342e302d6c69676874677265792e737667">
</pre>

<h1 style="text-align:center;">ATIVIDADE FINAL PARA ENTREGA EM GRUPO</h1>

    3) Reproduzir em código Python uma solução equivalente ao apresentado pelos pesquisadores do artigo, inclusive usando as mesmas técnicas, a fim de gerar uma solução para o problema em questão.
    4) Comentar o código, em especial os recursos de NLP usados para o processamento do texto e comparar os achados com eventuais resultados apresentados pelos autores.

|                 |              |
|:----------------|:-------------|
|**Universidade:**| Nove de Julho|
|**Programa:**    | PPGI - Programa de Pós Graduação em Informática |
|**Disciplina:**  | PROCESSAMENTO DE LINGUAGEM NATURAL - CONCEITOS E APLICAÇÕES|
|**Professor:**   | Prof. Dr. Cleber Gustavo Dias|
|**Alunos:**       | André L.M.F.Santos (__RA:622150026__); Maria Fátima B.Marques (__RA:622250106__); e Reinaldo R. Oliveira (__RA:622250107__)|

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\99836932\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# aponta o caminho onde estão os dados
os.chdir('C:/Users/99836932/Downloads/PPGI_Hotel/TopicModel')
# ler os dados
reviews_df = pd.read_csv('Hotel_Reviews.csv')
print(len(reviews_df))
print('Máximo Average Score =', max(reviews_df.Average_Score))
reviews_df.head()

515738
Máximo Average Score = 9.8


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
# Top 10 hotéis
top10 = reviews_df.groupby(['Hotel_Name'])['Average_Score'].mean().reset_index()
top10 = top10.sort_values(by = ['Average_Score'], ascending = False)
top10.head(10)

,Hotel_Name,Average_Score
1202,Ritz Paris,9.8
3,41,9.6
472,H tel de La Tamise Esprit de France,9.6
499,Haymarket Hotel,9.6
771,Hotel The Serras,9.6
481,H10 Casa Mimosa 4 Sup,9.6
598,Hotel Casa Camper,9.6
1015,Milestone Hotel Kensington,9.5
1303,Taj 51 Buckingham Gate Suites and Residences,9.5
750,Hotel Sacher Wien,9.5


In [4]:
# Cria uma coluna juntando os Reviews negativos e positivos
reviews_df["review"] = reviews_df["Negative_Review"] + reviews_df["Positive_Review"]
# cria uma coluna de rótulos de reviews ruins (1), zero caso contrário
reviews_df["is_bad_review"] = reviews_df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)
# seleciona apenas as colunas de interesse
reviews_df = reviews_df[["review", "is_bad_review", "Hotel_Name"]]
# Seleciona apenas se não for bad_review
reviews_df = reviews_df[reviews_df.is_bad_review == 0] # NPS Muito bom ou Excelente (acima de 5 pontos)
print(len(reviews_df))
reviews_df.head()

493457


,review,is_bad_review,Hotel_Name
1,No Negative No real complaints the hotel was g...,0,Hotel Arena
2,Rooms are nice but for elderly a bit difficul...,0,Hotel Arena
4,You When I booked with your company on line y...,0,Hotel Arena
5,Backyard of the hotel is total mess shouldn t...,0,Hotel Arena
7,Apart from the price for the brekfast Everyth...,0,Hotel Arena


In [5]:
# Concatena os reviews por hotel
reviews_df = reviews_df.groupby('Hotel_Name').agg({'review': lambda x: ''.join(x)}).reset_index()
reviews_df

,Hotel_Name,review
0,11 Cadogan Gardens,Thought the prise of drinks at the bar a litt...
1,1K Hotel,On a main street so can hear the traffic book...
2,25hours Hotel beim MuseumsQuartier,Breakfast not included and buffet really expe...
3,41,There wasn t a thing that we didn t like Its...
4,45 Park Lane Dorchester Collection,More kinds of fruit juice will make the mini ...
...,...,...
1487,citizenM London Bankside,This was our third stay at this hotel and it ...
1488,citizenM London Shoreditch,Lifts need reprogramming exasperating journey...
1489,citizenM Tower of London,Rooms are small but well designed The use of ...
1490,every hotel Piccadilly,The hotel overall requires an update furnitur...


In [6]:
# Confere o total de Hotéis na base
len(reviews_df.Hotel_Name.unique())

1492

In [7]:
# Cria uma dataframe para verificar se há algum hotel repetido
dh = pd.DataFrame(reviews_df.groupby(['Hotel_Name']).size()).reset_index()
dh.columns = ['Hotel','Total']
dh.sort_values(by = ['Total'], ascending = False)

,Hotel,Total
0,11 Cadogan Gardens,1
992,Mercure Paris Gare Montparnasse,1
1001,Mercure Paris Pigalle Sacre Coeur,1
1000,Mercure Paris Opera Louvre,1
999,Mercure Paris Opera Grands Boulevards,1
...,...,...
494,Ham Yard Hotel,1
493,Hallmark Hotel London Chigwell Prince Regent,1
492,HCC St Moritz,1
491,HCC Regente,1


In [8]:
# Amostra
#reviews_df = reviews_df.sample(frac = 0.01, replace = False, random_state=42)

# Limpeza dos dados

In [9]:
# remove 'No Negative' or 'No Positive' from text
reviews_df["review"] = reviews_df["review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [10]:
# Preprocessamento do corpus (reviews)
#import nltk
#import numpy as np

# Remove stop words em inglês
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

# Função para normalizar o corpus
def normalize_document(doc):
    # letras minúsculas e remover caracteres especiais\espaço em branco
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenizar documento
    tokens = wpt.tokenize(doc)
    # filtrar stopwords do documento
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # recriar documento a partir de tokens filtrados
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [ ]:
# Normalizar o documento
#import re
reviews_df["review_clean"] = normalize_corpus(reviews_df["review"])
print(len(reviews_df))
reviews_df.head()

# TF-IDF Features

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Converter os textos em vetores TF-IDF
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(reviews_df["review_clean"])

# Calcule a similaridade do cosseno entre os vetores
similarity = cosine_similarity(vectors)
similarity

In [ ]:
# Adcionar nome dos hotéis nas colunas
sim = pd.DataFrame(similarity)
new_labels = reviews_df.Hotel_Name.to_list()
sim = sim.set_axis(new_labels, axis=1)
sim.index = new_labels
sim

In [ ]:
# Analisa quais os hotéis com maior similaridade ao Hotel Pulizer
HP = sim[['Hotel Pulitzer']]
HP = HP.sort_values(by = ['Hotel Pulitzer'], ascending = False)
HP = HP.iloc[1: , :]
HP.head(10)

## Considerações finais

Os 3 primeiros hotéis do nosso modelo foram os mesmos do artigo; porém, a ordem dos outros ficou diferente.
Primeiro, isso aconteceu, pois atribuímos ao modelo uma nota de corte, com base no Net Promoter Score (NPS). Assim, só foram considerados as notas com NPS Muito bom ou Excelente (acima de 5 pontos).

Outra diferença pode ser em relação ao tratamento e limpeza dos dados, além da biblioteca que os autores usaram.

## Referências

* [Sentiment analysis with hotel reviews](https://www.kaggle.com/code/jonathanoheix/sentiment-analysis-with-hotel-reviews)
* [Github $-$ TopicModel Script](https://github.com/andrelmfsantos/Tese_COVID19/tree/main/TopicModel)
* [Net Promoter Score](https://resultadosdigitais.com.br/marketing/o-que-e-nps/)